In [ ]:
!pip3 install spacy==3.2.3

     |████████████████████████████████| 6.0 MB 12.9 MB/s 
     |████████████████████████████████| 451 kB 72.2 MB/s 
     |████████████████████████████████| 10.1 MB 59.3 MB/s 
     |████████████████████████████████| 42 kB 1.3 MB/s 
     |████████████████████████████████| 181 kB 71.2 MB/s 
     |████████████████████████████████| 653 kB 64.0 MB/s 
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Attempting uninstall: spacy
    Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4


In [ ]:
!python3.7 -m spacy download ru_core_news_md

     |████████████████████████████████| 43.0 MB 77.3 MB/s 
     |████████████████████████████████| 55 kB 2.6 MB/s 
     |████████████████████████████████| 8.2 MB 57.4 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_md')


In [ ]:
import spacy
from spacy import displacy

In [ ]:
nlp = spacy.load("ru_core_news_md")

### Функции извлечения информации

In [ ]:
def find_ner(doc):
  names = {}
  for ent in doc.ents:
      names[ent.text] = ent.label_
  return names

In [ ]:
def find_svo_triples(doc):

  svo_pairs = {verb.text: {} for verb in doc if verb.pos_ == 'VERB' and verb.dep_ != "xcomp" or verb.pos_ == 'AUX'}
  lemmatized_svo_pairs = {verb.lemma_: {} for verb in doc if verb.pos_ == 'VERB' and verb.dep_ != "xcomp" or verb.pos_ == 'AUX'}

  passive_voice = False

  for sentence in doc.sents:

    compound_predicats = {}

    for token in sentence:
      if 'subj:pass' in token.dep_:
        passive_voice = True

    for token in sentence:
      if not passive_voice:
        if token.dep_.endswith("xcomp") and token.pos_ == 'VERB' and token.head.pos_ in ['VERB', 'AUX']:
          # получаем составные глагольные сказуемые, потому что инфинитив - главное для объекта
          # если инфинитив зависит от инфинитива, мы пропускаем этот токен
          try:
            compound_predicats[token.head.text] = token.text
            svo_pairs[token.head.text]['comppred'] = token
            lemmatized_svo_pairs[token.head.lemma_]['comppred'] = token.lemma_
          except KeyError:
            continue
        elif token.pos_ == 'VERB' and not token.dep_.endswith("xcomp"):
          compound_predicats[token.text] = ''

        elif token.dep_.endswith("subj") and token.head.pos_ == 'VERB':
          # получаем субъект
          try:
            svo_pairs[token.head.text]['subj'] = token
            lemmatized_svo_pairs[token.head.lemma_]['subj'] = token.lemma_
          except KeyError:
            continue
          # получаем другие глаголы, у которых субъект тот же, и присваиваем им этот субъект
          # исключаем xcomp, потому что это инфинитивы
          verb_children = [child for child in token.head.children if child.pos_ == 'VERB']
          if verb_children:
            for child in verb_children:
              if child.dep_ != 'xcomp':
                svo_pairs[child.text]['subj'] = token
                lemmatized_svo_pairs[child.lemma_]['subj'] = token.lemma_
    
        elif token.dep_ == "obj" and token.head.pos_ == 'VERB':
          try:
          # получаем прямой объект предиката (записываем через ключ в зависимости от того, есть ли СГС)
            if token.head.text in compound_predicats.keys():
              svo_pairs[token.head.text]['obj'] = token
              lemmatized_svo_pairs[token.head.lemma_]['obj'] = token.lemma_
            elif token.head.text in compound_predicats.values():
              svo_pairs[token.head.head.text]['obj'] = token
              lemmatized_svo_pairs[token.head.head.lemma_]['obj'] = token.lemma_
          except KeyError:
            continue
      
        elif token.dep_ == 'iobj' and token.head.pos_ == 'VERB':
          if token.morph.get('Case')[0] == 'Dat':
            try:
          # получаем непрямой объект предиката (записываем через ключ в зависимости от того, есть ли СГС)
              if token.head.text in compound_predicats.keys():
                svo_pairs[token.head.text]['iobj'] = token
                lemmatized_svo_pairs[token.head.lemma_]['iobj'] = token.lemma_
              elif token.head.text in compound_predicats.values():
                svo_pairs[token.head.head.text]['iobj'] = token
                lemmatized_svo_pairs[token.head.head.lemma_]['iobj'] = token.lemma_
            except KeyError:
              continue

      else:
        # если пассивный залог
        if 'subj:pass' in token.dep_ and token.head.pos_ == 'VERB':
          svo_pairs[token.head.text]['obj'] = token
          lemmatized_svo_pairs[token.head.lemma_]['obj'] = token.lemma_
        elif token.dep_.endswith("obl") == True and token.head.pos_ == 'VERB':
          prep = [child for child in token.children if child.dep_ == 'case']
          if prep:
            continue
          else:
            try:
              svo_pairs[token.head.text]['subj'] = token
              lemmatized_svo_pairs[token.head.lemma_]['subj'] = token.lemma_
            except KeyError:
              continue

  return svo_pairs, lemmatized_svo_pairs

In [ ]:
def find_relations_ner(doc, names):
  relations_for_ner = {}
  for sentence in doc.sents:
    for token in sentence:
      if token.text in names.keys() and "subj" not in token.dep_:
        relations_for_ner[token.head.text] = token
  return relations_for_ner

### Создание датасета

In [ ]:
import os
import pandas as pd

In [ ]:
list_of_files = []
for filename in os.listdir(r"stories"):
    list_of_files.append(filename)

In [ ]:
list_of_files = sorted(list_of_files, key=lambda x: int(x.split('.')[0]))
list_of_files

['1.txt',
 '2.txt',
 '3.txt',
 '4.txt',
 '5.txt',
 '6.txt',
 '7.txt',
 '8.txt',
 '9.txt',
 '10.txt',
 '11.txt',
 '12.txt',
 '13.txt',
 '14.txt',
 '15.txt',
 '16.txt',
 '17.txt',
 '18.txt',
 '19.txt',
 '20.txt',
 '21.txt',
 '22.txt',
 '23.txt',
 '24.txt',
 '25.txt',
 '26.txt',
 '27.txt',
 '28.txt',
 '29.txt',
 '30.txt']

In [ ]:
df = pd.DataFrame(columns=["text", "story_id", "doc", "ner", "svo_triples", "svo_lemmas", "relations"])

for filename in list_of_files:
    text = open(os.path.join("stories", filename), encoding='utf8', errors='ignore').read()
    label = filename.split('.')[0]
    df = df.append({
                    "text": text,
                    "story_id":  label
                    }, ignore_index=True)
    
df

,text,story_id,doc,ner,svo_triples,svo_lemmas,relations
0,Скажу вам откровенно: я очень люблю людей.\n\n...,1,NaN,NaN,NaN,NaN,NaN
1,Один мальчик гулял в лесу и нашел гнездышко.\n...,2,NaN,NaN,NaN,NaN,NaN
2,"Танюша много слышала о черенках, а что это так...",3,NaN,NaN,NaN,NaN,NaN
3,Бережливым мальчиком Костя рос. Даст ему мать ...,4,NaN,NaN,NaN,NaN,NaN
4,"Как-то в мае, когда снег уже потаял, я сидел н...",5,NaN,NaN,NaN,NaN,NaN
5,А сейчас не весна — сейчас поздняя осень. В ок...,6,NaN,NaN,NaN,NaN,NaN
6,"У Бобки были замечательные штаны: зеленые, вер...",7,NaN,NaN,NaN,NaN,NaN
7,Мы с Валей затейники. Мы всегда затеваем какие...,8,NaN,NaN,NaN,NaN,NaN
8,— А сейчас я покажу вам Шанго. Ему сорок лет. ...,9,NaN,NaN,NaN,NaN,NaN
9,"Когда я был на Дальнем Востоке, то долго думал...",10,NaN,NaN,NaN,NaN,NaN


#### Проверка работы на 1 тексте

In [ ]:
doc_1_text = nlp(df['text'][0])
names_1_text = find_ner(doc_1_text)
names_1_text

{'Алупки': 'LOC',
 'Алупку': 'LOC',
 'Гляжу': 'PER',
 'Крыму': 'LOC',
 'Ленинград': 'LOC',
 'Направо': 'PER',
 'Ходко': 'PER',
 'Шагов': 'PER',
 'Ялты': 'LOC'}

In [ ]:
df['ner'][0] = names_1_text

In [ ]:
svo_1_text, lemmas_1_text = find_svo_triples(doc_1_text)
df['svo_triples'][0] = svo_1_text
df['svo_lemmas'][0] = lemmas_1_text

In [ ]:
relations_1_text = find_relations_ner(doc_1_text, df['ner'][0])
df['relations'][0] = relations_1_text
df

,text,story_id,doc,ner,svo_triples,svo_lemmas,relations
0,Скажу вам откровенно: я очень люблю людей.\n\n...,1,NaN,"{'Ялты': 'LOC', 'Алупку': 'LOC', 'Крыму': 'LOC...","{'Скажу': {'iobj': вам}, 'люблю': {'subj': я, ...","{'сказать': {'iobj': 'вам'}, 'любить': {'subj'...","{'знаете': Алупку, 'был': Крыму, 'знает': Алуп..."
1,Один мальчик гулял в лесу и нашел гнездышко.\n...,2,NaN,NaN,NaN,NaN,NaN
2,"Танюша много слышала о черенках, а что это так...",3,NaN,NaN,NaN,NaN,NaN
3,Бережливым мальчиком Костя рос. Даст ему мать ...,4,NaN,NaN,NaN,NaN,NaN
4,"Как-то в мае, когда снег уже потаял, я сидел н...",5,NaN,NaN,NaN,NaN,NaN
5,А сейчас не весна — сейчас поздняя осень. В ок...,6,NaN,NaN,NaN,NaN,NaN
6,"У Бобки были замечательные штаны: зеленые, вер...",7,NaN,NaN,NaN,NaN,NaN
7,Мы с Валей затейники. Мы всегда затеваем какие...,8,NaN,NaN,NaN,NaN,NaN
8,— А сейчас я покажу вам Шанго. Ему сорок лет. ...,9,NaN,NaN,NaN,NaN,NaN
9,"Когда я был на Дальнем Востоке, то долго думал...",10,NaN,NaN,NaN,NaN,NaN


#### Обработка всех текстов

In [ ]:
df["doc"] = df["text"].apply(nlp)

In [ ]:
df["ner"] = df["doc"].apply(find_ner)

In [ ]:
for ind in range(len(list_of_files)):
  svo_triples, svo_lemmas = find_svo_triples(df["doc"][ind])
  df["svo_triples"][ind] = svo_triples
  df["svo_lemmas"][ind] = svo_lemmas
  relations = find_relations_ner(df["doc"][ind], df['ner'][ind])
  df['relations'][ind] = relations

df.head()

,text,story_id,doc,ner,svo_triples,svo_lemmas,relations
0,Скажу вам откровенно: я очень люблю людей.\n\n...,1,"(Скажу, вам, откровенно, :, я, очень, люблю, л...","{'Ялты': 'LOC', 'Алупку': 'LOC', 'Крыму': 'LOC...","{'Скажу': {'iobj': вам}, 'люблю': {'subj': я, ...","{'сказать': {'iobj': 'вам'}, 'любить': {'subj'...","{'знаете': Алупку, 'был': Крыму, 'знает': Алуп..."
1,Один мальчик гулял в лесу и нашел гнездышко.\n...,2,"(Один, мальчик, гулял, в, лесу, и, нашел, гнез...",{},"{'гулял': {'subj': мальчик}, 'нашел': {'subj':...","{'гулять': {'subj': 'мальчик'}, 'найти': {'sub...",{}
2,"Танюша много слышала о черенках, а что это так...",3,"(Танюша, много, слышала, о, черенках, ,, а, чт...","{'Танюша': 'PER', 'Таня': 'PER', 'Танюше': 'PE...","{'слышала': {'subj': Танюша}, 'знала': {'subj'...","{'слышать': {'subj': 'танюша'}, 'знать': {'sub...","{'Будем': Танюша, 'принялась': Танюша, 'порабо..."
3,Бережливым мальчиком Костя рос. Даст ему мать ...,4,"(Бережливым, мальчиком, Костя, рос, ., Даст, е...","{'Костя': 'PER', 'Федя': 'PER', 'Шел Костя': '...","{'рос': {'subj': Костя}, 'Даст': {'iobj': ему,...","{'расти': {'subj': 'костя'}, 'даст': {'iobj': ...","{'дружок': Федя, 'Костя': Костя, 'Шел': Костя,..."
4,"Как-то в мае, когда снег уже потаял, я сидел н...",5,"(Как, -, то, в, мае, ,, когда, снег, уже, пота...","{'Зуй': 'PER', 'Витьки Белова': 'PER', 'Зуюшки...","{'сидел': {'subj': Дядя}, 'вынесенном': {}, 'ч...","{'сидеть': {'subj': 'дядя'}, 'вынести': {}, 'ч...","{'Дядя': Зуй, 'дядя': Зуй, 'бродили': Зуюшки, ..."


### Формирование датафреймов с извлеченными данными

#### Работа с глаголами с объектами

In [ ]:
df_lemmas = pd.DataFrame(df["svo_lemmas"])
df_lemmas

,svo_lemmas
0,"{'сказать': {'iobj': 'вам'}, 'любить': {'subj'..."
1,"{'гулять': {'subj': 'мальчик'}, 'найти': {'sub..."
2,"{'слышать': {'subj': 'танюша'}, 'знать': {'sub..."
3,"{'расти': {'subj': 'костя'}, 'даст': {'iobj': ..."
4,"{'сидеть': {'subj': 'дядя'}, 'вынести': {}, 'ч..."
5,"{'закрыть': {'subj': 'он', 'obj': 'земля'}, 'т..."
6,"{'быть': {'subj': 'штанов'}, 'сказать': {'obj'..."
7,"{'затевать': {'subj': 'мы', 'obj': 'игра'}, 'ч..."
8,"{'показать': {'subj': 'я', 'iobj': 'вам', 'obj..."
9,"{'был': {}, 'думать': {}, 'бы': {}, 'привезти'..."


In [ ]:
def get_svo_verbs(dict_verbs):
  svo_verbs = []
  for verb in dict_verbs.keys():
    if dict_verbs[verb].keys():
      svo_verbs.append(verb)
  return svo_verbs

In [ ]:
def get_verbs_with_obj(dict_verbs):
  svo_verbs = []
  for verb in dict_verbs.keys():
    if 'obj' in dict_verbs[verb].keys():
      svo_verbs.append((verb, dict_verbs[verb]))
  return svo_verbs

In [ ]:
def get_verbs_with_iobj(dict_verbs):
  svo_verbs = []
  for verb in dict_verbs.keys():
    if 'iobj' in dict_verbs[verb].keys():
      svo_verbs.append((verb, dict_verbs[verb]))
  return svo_verbs

In [ ]:
df_lemmas["keys"] = df_lemmas["svo_lemmas"].apply(get_svo_verbs)
df_lemmas["verbs_with_obj"] = df_lemmas["svo_lemmas"].apply(get_verbs_with_obj)
df_lemmas["verbs_with_iobj"] = df_lemmas["svo_lemmas"].apply(get_verbs_with_iobj)
df_lemmas.head()

,svo_lemmas,keys,verbs_with_obj,verbs_with_iobj
0,"{'сказать': {'iobj': 'вам'}, 'любить': {'subj'...","[сказать, любить, знать, растрачивать, купать,...","[(любить, {'subj': 'я', 'obj': 'человек', 'com...","[(сказать, {'iobj': 'вам'}), (дать, {'subj': '..."
1,"{'гулять': {'subj': 'мальчик'}, 'найти': {'sub...","[гулять, найти, сидеть, пищать, прилететь, пок...","[(найти, {'subj': 'мальчик', 'obj': 'птенчик'}...","[(ушибить, {'subj': 'птичка', 'iobj': 'себя', ..."
2,"{'слышать': {'subj': 'танюша'}, 'знать': {'sub...","[слышать, знать, стать, удивиться, вырасти, не...","[(стать, {'subj': 'таня', 'comppred': 'разгляд...",[]
3,"{'расти': {'subj': 'костя'}, 'даст': {'iobj': ...","[расти, даст, положить, появиться, случиться, ...","[(положить, {'subj': 'костя', 'obj': 'денежка'...","[(даст, {'iobj': 'ему', 'subj': 'мать'}), (пок..."
4,"{'сидеть': {'subj': 'дядя'}, 'вынести': {}, 'ч...","[сидеть, чистил, заворачивал, видеть, сказать,...","[(чистил, {'subj': 'я', 'obj': 'ружьё'}), (зав...",[]


In [ ]:
df_lemmas.to_excel('verbs_dictionaries.xlsx', sheet_name='verbs', index=False)

In [ ]:
verbs_keys = list(df_lemmas["keys"])
verbs_lemmas = []
for verbs_list in verbs_keys:
  verbs_lemmas.extend(verbs_list)

In [ ]:
freq_verbs_svo = {}
for verb in verbs_lemmas:
  freq_verbs_svo[verb] = freq_verbs_svo.get(verb, 0) + 1

print(len(freq_verbs_svo.keys()))
sort_freq = sorted(freq_verbs_svo.items(), key=lambda x: x[1], reverse=True)
sort_freq[:5]

773


[('сказать', 19), ('стать', 17), ('пойти', 14), ('хотеть', 14), ('мочь', 13)]

In [ ]:
freq_verbs_list = []
for verb in sort_freq:
  freq_verbs_list.append(verb[0])
freq_verbs_list[:5]

['сказать', 'стать', 'пойти', 'хотеть', 'мочь']

In [ ]:
df_verbs_with_obj = df_lemmas["verbs_with_obj"]
df_verbs_with_iobj = df_lemmas["verbs_with_iobj"]
df_verbs_with_obj.to_excel('verbs_dir_object.xlsx', sheet_name='direct object', index=False)
df_verbs_with_iobj.to_excel('verbs_indir_object.xlsx', sheet_name='indirect object', index=False)

In [ ]:
with open("verbs.txt", "w") as file:
  print(*sort_freq, file=file, sep="\n")

#### Работа с именованными сущностями

In [ ]:
df_ner = pd.DataFrame(df["ner"])
df_ner

,ner
0,"{'Ялты': 'LOC', 'Алупку': 'LOC', 'Крыму': 'LOC..."
1,{}
2,"{'Танюша': 'PER', 'Таня': 'PER', 'Танюше': 'PE..."
3,"{'Костя': 'PER', 'Федя': 'PER', 'Шел Костя': '..."
4,"{'Зуй': 'PER', 'Витьки Белова': 'PER', 'Зуюшки..."
5,"{'Пантелевна': 'PER', 'Дрова': 'PER', 'Пантеле..."
6,"{'Бобки': 'LOC', 'Бобка': 'PER'}"
7,"{'Валей': 'PER', 'Три': 'LOC', 'Валя': 'PER', ..."
8,"{'Шанго': 'PER', 'Иван Акимович': 'PER'}"
9,"{'Дальнем Востоке': 'LOC', 'Маленький': 'LOC',..."


In [ ]:
df_ner["ner_type"] = df_ner["ner"].apply(lambda d: d.items())
df_ner["ner_type"] = df_ner["ner_type"].apply(lambda item: list(item))
df_ner.head()

,ner,ner_type
0,"{'Ялты': 'LOC', 'Алупку': 'LOC', 'Крыму': 'LOC...","[(Ялты, LOC), (Алупку, LOC), (Крыму, LOC), (На..."
1,{},[]
2,"{'Танюша': 'PER', 'Таня': 'PER', 'Танюше': 'PE...","[(Танюша, PER), (Таня, PER), (Танюше, PER), (Т..."
3,"{'Костя': 'PER', 'Федя': 'PER', 'Шел Костя': '...","[(Костя, PER), (Федя, PER), (Шел Костя, PER), ..."
4,"{'Зуй': 'PER', 'Витьки Белова': 'PER', 'Зуюшки...","[(Зуй, PER), (Витьки Белова, PER), (Зуюшки, PE..."


In [ ]:
ner_all = list(df_ner["ner_type"])
ner_pairs = []
for pair in ner_all:
  ner_pairs.extend(pair)

ner_pairs[:5]

[('Ялты', 'LOC'),
 ('Алупку', 'LOC'),
 ('Крыму', 'LOC'),
 ('Направо', 'PER'),
 ('Алупки', 'LOC')]

#### Работа с отношениями

In [ ]:
df_relations = pd.DataFrame(df["relations"])
df_relations

,relations
0,"{'знаете': Алупку, 'был': Крыму, 'знает': Алуп..."
1,{}
2,"{'Будем': Танюша, 'принялась': Танюша, 'порабо..."
3,"{'дружок': Федя, 'Костя': Костя, 'Шел': Костя,..."
4,"{'Дядя': Зуй, 'дядя': Зуй, 'бродили': Зуюшки, ..."
5,"{'мы': Пантелевной, 'Вон': Пантелевна, 'Пантел..."
6,{'были': Бобки}
7,"{'Мы': Валей, 'поросёнка': Три, 'Давай': Петя,..."
8,"{'покажу': Шанго, 'привезли': Шанго, 'Догнал':..."
9,"{'Маленький': Маленький, 'Приготовили': Симпат..."


In [ ]:
df_relations["relation_pairs"] = df_relations["relations"].apply(lambda d: d.items())
df_relations["relation_pairs"] = df_relations["relation_pairs"].apply(lambda item: list(item))
df_relations.head()

,relations,relation_pairs
0,"{'знаете': Алупку, 'был': Крыму, 'знает': Алуп...","[(знаете, Алупку), (был, Крыму), (знает, Алупк..."
1,{},[]
2,"{'Будем': Танюша, 'принялась': Танюша, 'порабо...","[(Будем, Танюша), (принялась, Танюша), (порабо..."
3,"{'дружок': Федя, 'Костя': Костя, 'Шел': Костя,...","[(дружок, Федя), (Костя, Костя), (Шел, Костя),..."
4,"{'Дядя': Зуй, 'дядя': Зуй, 'бродили': Зуюшки, ...","[(Дядя, Зуй), (дядя, Зуй), (бродили, Зуюшки), ..."


In [ ]:
relations_all = list(df_relations["relation_pairs"])
relations_pairs = []
for pair in relations_all:
  relations_pairs.extend(pair)

relations_pairs[:10]

[('знаете', Алупку),
 ('был', Крыму),
 ('знает', Алупки),
 ('дойти', Алупки),
 ('Алупку', Алупку),
 ('пришли', Алупку),
 ('вернувшись', Ленинград),
 ('Будем', Танюша),
 ('принялась', Танюша),
 ('поработать', Тане)]

In [ ]:
with open("relations.txt", "w") as file:
  print(*relations_pairs, file=file, sep="\n")